In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, T5Tokenizer

In [3]:
# directory where to cache models
cache_dir = '/home/jupyter/data/transformers'
checkpoint = "lmsys/fastchat-t5-3b-v1.0"

In [4]:
# loading model and tokenizer based on particular model checkpoint
# tokenizer is used to preprocess text input in a way that the model can understand
tokenizer = T5Tokenizer.from_pretrained(checkpoint, device_map="auto", use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, device_map="auto", low_cpu_mem_usage=True, 
                                              cache_dir=cache_dir, torch_dtype=torch.float16)

In [5]:
system="""A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions.
### Human: What are the key differences between renewable and non-renewable energy sources?
### Assistant: Renewable energy sources are those that can be replenished naturally in a relatively short amount of time, such as solar, wind, hydro, geothermal, and biomass. Non-renewable energy sources, on the other hand, are finite and will eventually be depleted, such as coal, oil, and natural gas. Here are some key differences between renewable and non-renewable energy sources:
1. Availability: Renewable energy sources are virtually inexhaustible, while non-renewable energy sources are finite and will eventually run out.
2. Environmental impact: Renewable energy sources have a much lower environmental impact than non-renewable sources, which can lead to air and water pollution, greenhouse gas emissions, and other negative effects.
3. Cost: Renewable energy sources can be more expensive to initially set up, but they typically have lower operational costs than non-renewable sources.
4. Reliability: Renewable energy sources are often more reliable and can be used in more remote locations than non-renewable sources.
5. Flexibility: Renewable energy sources are often more flexible and can be adapted to different situations and needs, while non-renewable sources are more rigid and inflexible.
6. Sustainability: Renewable energy sources are more sustainable over the long term, while non-renewable sources are not, and their depletion can lead to economic and social instability."""

system = system + "\n"


def generate(prompt, max_new_tokens=1024, temperature=0.7):
    torch.cuda.empty_cache()

    template = system + "### Human: %s \n"
    input_start = template % prompt
    input_final = template % prompt + "### Assistant:"
    
    inputs = tokenizer(input_final, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, 
                             temperature=temperature)
    result = tokenizer.batch_decode(outputs, skip_special_tokens=True, spaces_between_special_tokens=False)
        
    return result[0]

In [6]:
prompt = "translate English to German: How old are you?"

response = generate(prompt)
print(response)

Ja, wie alt sind Sie?



In [7]:
prompt = "How to make a BLT?"

response = generate(prompt)
print(response)

A BLT (British Tuna Lettuce Sandwich) is a classic sandwich that consists of two slices of bread, two slices of tomato, and a slice of lettuce. Here is a step-by-step guide to making a BLT:
1. Toast the bread: Toast the bread on a grill or in a toaster until it is golden brown and crispy.
2. Spread the mayonnaise on one slice of bread.
3. Spread the mustard on the other slice of bread.
4. Place the tomato slice on top of the lettuce slice.
5. Place the slice of bread on top of the mayonnaise slice.
6. Place the slice of bread on top of the lettuce slice.
7. Place the slice of bread on top of the mayonnaise slice.
8. Place the slice of bread on top of the lettuce slice.
9. Place the slice of bread on top of the mayonnaise slice.
10. Place the slice of bread on top of the mayonnaise slice.
11. Place the slice of bread on top of the lettuce slice.
12. Place the slice of bread on top of the mayonnaise slice.
13. Place the slice of bread on top of the lettuce slice.
14. Place the slice of b

In [8]:
prompt = "A step by step recipe to make mapo tofu without repeating steps:"

response = generate(prompt)
print(response)

Here is a step-by-step recipe for making mapo tofu, which is a popular dish in Taiwan, using a variety of ingredients and cooking methods:
1. Rinse the tofu in a colander and drain it thoroughly.
2. In a large pot, heat the oil over medium heat. Add the garlic, ginger, and onion and cook until the onion is translucent and the ginger is fragrant, about 5 minutes.
3. Add the tofu to the pot and stir-fry until the tofu is cooked through, about 5 minutes.
4. Add the soy sauce, sugar, and water to the pot and bring to a boil. Reduce the heat to low and simmer for about 10 minutes, or until the sauce has thickened.
5. Add the sliced tofu to the pot and stir-fry until the tofu is heated through, about 5 minutes.
6. Remove the tofu from the pot and set it aside to cool.
7. In a large skillet, heat the oil over medium heat. Add the garlic, ginger, and onion and cook until the onion is translucent and the ginger is fragrant, about 5 minutes.
8. Add the tofu to the skillet and stir-fry until the 

In [9]:
prompt = "You are an equity researcher specialized in the health and pharmaceutical industry. Write a summary for the following article. Focus on the information that will impact investment decisions on the companies mentioned."
article = """Alzheimer’s disease treatment stocks are focused on Alzheimer’s disease, a degenerative brain disorder that results in declining memory and thinking skills and typically affects people in their mid-60s.According to the Alzheimer’s Association, neurons in other areas of the brain also begin to deteriorate as Alzheimer’s disease gets worse, resulting in the loss of basic human functions and overall cognitive impairment.This condition affects more than 6.7 million people in the US alone; it’s also the most common form of dementia and is the sixth leading cause of death in America. Treatments are available to alleviate Alzheimer’s disease symptoms, but because they do not affect the underlying causes of this neurodegenerative disease, they’re only a bandaid solution.Therapies approved by the US Food and Drug Administration (FDA) include: rivastigmine by Novartis (NYSE:NVS); galantamine, developed by Janssen, a division of Johnson & Johnson (NYSE:JNJ); donepezil by Pfizer (NYSE:PFE); and AbbVie's (NYSE:ABBV) memantine.Since there is no cure for Alzheimer’s disease, death is often the result for patients as the ailment devastates the brain. And unfortunately, Alzheimer’s disease is rising in prevalence — a report from Research and Markets suggests that the global Alzheimer’s disease treatment market will be worth a significant US$9.64 billion by 2028 as more patients need treatment, and as more investments are made in biomarkers for diagnosis and drug development.Here the Investing News Network takes a look at a few of the biggest Alzheimer’s disease treatment stocks on the NASDAQ. Companies are listed in order of market cap from largest to smallest, and all data was current as of April 19, 2023.. Biogen (NASDAQ:BIIB) Company Profile.Market cap: US$42.27 billion; current share price: US$292.80.Massachusetts-based Biogen, a pioneer in the field of neuroscience, is developing, manufacturing and marketing therapies aimed at treating serious neurological, neurodegenerative, autoimmune and rare diseases.The global biotechnology firm’s research areas include Alzheimer's disease and dementia. However, Biogen’s launch of its FDA-approved Alzheimer’s disease drug Aduhelm faced a lot of pushback in 2022, both from the market and from Congress, over what was viewed as a hasty fast-track approval process and exorbitant costs to patients.In early 2023, Biogen is looking to give it another go, this time with LEQEMBI (lecanemab-irmb) for the treatment of Alzheimer's disease, which the FDA has approved under its accelerated approval pathway. The drug was jointly developed by Biogen and Tokyo-based pharmaceutical company Eisai (OTC Pink:ESALF,TSE:4523). Intra-Cellular Therapies (NASDAQ:ITCI).Company Profile.Market cap: US$5.96 billion; current share price: US$62.59.As its name suggests, biopharma company Intra-Cellular Therapies is taking an intracellular approach to developing therapies for patients with complex psychiatric and neurologic diseases, including Parkinson’s disease and Alzheimer’s disease.Intra-Cellular Therapies is currently evaluating ITI-1284-ODT-SL, a deuterated form of lumateperone that comes in an oral disintegrating tablet; ITI-1284-ODT-SL is in Phase 1 studies, including drug-drug interaction studies. The company expects to commence Phase 2 clinical trials in agitation in patients with probable Alzheimer’s disease in 2023. 3. Axsome Therapeutics (NASDAQ:AXSM).Company Profile.Market cap: US$3.16 billion; current share price: US$72.64.New York-based Axsome Therapeutics is developing and commercializing a portfolio of novel therapies for disorders of the central nervous system with limited treatment options. Products currently on the market from the biopharmaceutical company include Auvelity, a rapid-acting oral antidepressant for the treatment of major depressive disorder; and Sunosi, a once-daily prescription medicine to treat excessive daytime sleepiness due to obstructive sleep apnea or narcolepsy.Axsome’s product pipeline includes AXS-05, a NMDA receptor antagonist with multimodal activity. It has breakthrough therapy designation from the FDA and is currently in Phase 3 clinical studies for the treatment of Alzheimer’s disease agitation. 
"""

response = generate(prompt + article)
print(response)

Token indices sequence length is longer than the specified maximum sequence length for this model (2290 > 2048). Running this sequence through the model will result in indexing errors


"The article discusses the importance of Alzheimer's disease treatment stocks in the health and pharmaceutical industry. The article highlights the prevalence of Alzheimer's disease in the US and the growing global Alzheimer's disease treatment market. The article also highlights the challenges faced by Biogen in launching its Alzheimer's disease drug Aduhelm and the intracellular approach taken by Intra-Cellular Therapies and Axsome Therapeutics. The article also provides information on the current market cap of the companies mentioned and their current share prices. The article provides insights into the potential investment opportunities in these companies.\n"

In [10]:
article = """
April 26 (Reuters) - The U.S. health regulator on Wednesday approved Seres Therapeutics Inc's (MCRB.O) pill for treating a type of bacterial infection, giving an easier and standardized option to patients who often have to rely on individual donors for fecal transplants.

The drug, branded as Vowst, was approved for treating 18 years and older for recurrent Clostridioides difficile (C. diff) infections (CDI), generally caused by prolonged use of antibiotics, which wipe out friendly colon bacteria and can cause potentially fatal diarrhea and inflammation of the colon.

Vowst consists of a certain kind of bacteria made by purifying fecal matter derived from healthy people, while fecal transplants are donated by healthy volunteers and are not purified.

The approval for the drug was based on its late-stage trials, in which treatment with the drug helped patients get rid of recurrent CDI.

"The efficacy is so good, I don't know that there's much room for others to compete with this drug," Oppenheimer analyst Mark Breidenbach said before the approval.

Seres' drug will compete with Switzerland-based Ferring Pharmaceuticals' fecal transplant-based therapy, which is delivered through an enema.

Physicians think Seres' pill could replace fecal microbiota transplants in patients, TD Cowen analyst Joseph Thome said ahead of the decision. He said the drug could be priced close to $20,000 per course, expecting peak sales of $750 million in the U.S. in 2033.

Government data estimates that C. diff causes nearly half a million infections in the United States annually.

Seres entered into a deal with Nestle's health science unit in 2021 to jointly commercialize the drug in the United States.
"""

response = generate(prompt + article)
print(response)

"The U.S. health regulator has approved Seres Therapeutics' (MCRB.O) pill for treating a type of bacterial infection, giving patients an easier and standardized option to fecal transplants. The drug, branded as Vowst, is made by purifying fecal matter derived from healthy people, while fecal transplants are donated by healthy volunteers and are not purified. The approval for the drug was based on its late-stage trials, in which treatment with the drug helped patients get rid of recurrent CDI. Seres' drug will compete with Switzerland-based Ferring Pharmaceuticals' fecal transplant-based therapy, which is delivered through an enema. Physicians think Seres' pill could replace fecal microbiota transplants in patients, and analysts expect peak sales of $750 million in the U.S. in 2033. The drug could be priced close to $20,000 per course, with peak sales of $750 million in the U.S. expected in 2033. Seres entered into a deal with Nestle's health science unit in 2021 to jointly commercializ

In [11]:
article = """
LONDON, April 28 (Reuters) - Pfizer Inc (PFE.N) has pledged to deliver critical new medicines more quickly in low-income nations, but its first such vaccine effort faces hurdles likely to delay distribution in poorer countries by several years, global health officials told Reuters.

Pfizer made a commitment on more equitable access last year, following criticism that it prioritized wealthy nations for doses of its COVID-19 shot early in the pandemic. The company says it wants to shorten a timeline in which poorer countries often get vaccines many years after they are available elsewhere.

The drugmaker received a $28 million grant from the Bill and Melinda Gates Foundation in September to support the launch of the respiratory syncytial virus vaccine in poorer countries, where RSV - a common cold-like virus - is much more likely to be lethal for very young children.

Pfizer's maternal RSV vaccine is expected to be approved for use in pregnant women in August in the United States and several months later in Europe, and the company plans to launch the product in both markets in the fall. Its peak sales forecast is of more than $2 billion annually together from the maternal vaccine and an RSV shot for older adults.

But the vaccine will need a different delivery system to be used in developing countries in Africa and Asia, including alternative packaging and syringes. Preparations for those modifications are just beginning, according to the World Health Organization and the company - which is likely to push back delivery by several years.

The case highlights how equitable global access will require better advance planning by drugmakers, governments and health organizations, health officials say.

"They could have tried sooner," said Erin Sparrow, WHO's technical officer for the RSV vaccine, referring to Pfizer. "It's a little bit disappointing, but they are doing the right thing now."

Pfizer says it is committed to a faster timeline for rolling out medicines to poor nations but acknowledges that there are challenges in meeting such goals.

"We are committed to working with the appropriate organizations, including regulatory authorities and other global health partners, to help ensure the vaccine candidate, once approved, is available in lower- and middle-income countries as quickly as possible," a Pfizer spokesperson said when asked about the timeline for the RSV shot.

In an interview last month, Pfizer's head of vaccines research and development Annaliesa Anderson said the company still aimed to make its vaccine – the first of its kind – available for pregnant women in low-income countries more or less in the "same time frame" as in wealthy countries.

It had pledged last May to offer its existing portfolio, as well as newly developed drugs and vaccines, at a not-for-profit basis to 45 lower-income countries on a faster timeline than in the past.

The company had learned during the COVID-19 pandemic that "parallel development" to meet the needs of different regions was possible, Anderson said.

Rates of RSV have surged since the pandemic. Inoculating pregnant women against RSV can protect babies from the virus that kills nearly 100,000 children worldwide each year.

Most of those children are under five years old, with 98% living in low and middle-income countries that have fewer healthcare resources to treat the infection.

Pfizer has yet to take a number of steps needed to make the vaccine available in developing countries, according to global health officials and the company.

They include seeking approval for the shot from the WHO, whose authorization is used by governments that don't have a strong domestic drugs regulator, and by nonprofits - such as the global vaccine alliance Gavi - that fund purchases in low-income countries.

Early discussions with the WHO are underway, but the process will be complicated because Pfizer's RSV vaccine is not designed for syringes that automatically disable after one injection, which are preferred by the health agency for safety reasons to prevent reuse.

In addition, the RSV shot is currently only available in a single-dose vial. In lower-resource settings, multi-dose vials are often used because they are cheaper and simpler to distribute.

Multi-dose vials require the addition of a preservative, which means the manufacturer needs to run new trials to ensure there is no impact on the vaccine’s safety or efficacy.

The Gates grant will be used to fund trials of a multi-dose vial, which are likely to begin next year and may take another year to complete, a source with knowledge of the plans said.

A Gates Foundation official said that the investment in the RSV program represented a learning process for the organization and the company on the steps needed for more equitable access.

They have already reached a partnership for a maternal vaccine in earlier stages of development against Group B Streptococcus, which kills around 90,000 babies a year and causes 46,000 stillbirths, over half of them in sub-Saharan Africa.

"We very much want to see that time gap [between high and low-income countries] reduced, such that there's no space between the two," said Padmini Srikantiah, lead on the RSV vaccine launch at Gates.

She still expects it to be several years before the RSV vaccine is launched in lower-income countries.

"I think we're doing better than in years before," she said. "It's a process."

(This story has been corrected to change the approval date in the US to August, not May, in paragraph 4)
"""

response = generate(prompt + article)
print(response)

"Pfizer has made a commitment to deliver critical new medicines more quickly in low-income nations, but its first such vaccine effort faces hurdles likely to delay distribution in poorer countries by several years. The company received a $28 million grant from the Bill and Melinda Gates Foundation in September to support the launch of the respiratory syncytial virus vaccine in poorer countries, where RSV is much more likely to be lethal for very young children. Pfizer has yet to take a number of steps needed to make the vaccine available in developing countries, including seeking approval for the shot from the WHO, whose authorization is used by governments that don't have a strong domestic drugs regulator, and by nonprofits - such as the global vaccine alliance Gavi. The Gates grant will be used to fund trials of a multi-dose vial, which are likely to begin next year and may take another year to complete. The company has already reached a partnership for a maternal vaccine in earlier 

In [12]:
article = """
LONDON, April 28 (Reuters) - Pfizer Inc (PFE.N) has pledged to deliver critical new medicines more quickly in low-income nations, but its first such vaccine effort faces hurdles likely to delay distribution in poorer countries by several years, global health officials told Reuters.

Pfizer made a commitment on more equitable access last year, following criticism that it prioritized wealthy nations for doses of its COVID-19 shot early in the pandemic. The company says it wants to shorten a timeline in which poorer countries often get vaccines many years after they are available elsewhere.

The drugmaker received a $28 million grant from the Bill and Melinda Gates Foundation in September to support the launch of the respiratory syncytial virus vaccine in poorer countries, where RSV - a common cold-like virus - is much more likely to be lethal for very young children.

Pfizer's maternal RSV vaccine is expected to be approved for use in pregnant women in August in the United States and several months later in Europe, and the company plans to launch the product in both markets in the fall. Its peak sales forecast is of more than $2 billion annually together from the maternal vaccine and an RSV shot for older adults.

But the vaccine will need a different delivery system to be used in developing countries in Africa and Asia, including alternative packaging and syringes. Preparations for those modifications are just beginning, according to the World Health Organization and the company - which is likely to push back delivery by several years.

The case highlights how equitable global access will require better advance planning by drugmakers, governments and health organizations, health officials say.

"They could have tried sooner," said Erin Sparrow, WHO's technical officer for the RSV vaccine, referring to Pfizer. "It's a little bit disappointing, but they are doing the right thing now."

Pfizer says it is committed to a faster timeline for rolling out medicines to poor nations but acknowledges that there are challenges in meeting such goals.

"We are committed to working with the appropriate organizations, including regulatory authorities and other global health partners, to help ensure the vaccine candidate, once approved, is available in lower- and middle-income countries as quickly as possible," a Pfizer spokesperson said when asked about the timeline for the RSV shot.

In an interview last month, Pfizer's head of vaccines research and development Annaliesa Anderson said the company still aimed to make its vaccine – the first of its kind – available for pregnant women in low-income countries more or less in the "same time frame" as in wealthy countries.

It had pledged last May to offer its existing portfolio, as well as newly developed drugs and vaccines, at a not-for-profit basis to 45 lower-income countries on a faster timeline than in the past.

The company had learned during the COVID-19 pandemic that "parallel development" to meet the needs of different regions was possible, Anderson said.

Rates of RSV have surged since the pandemic. Inoculating pregnant women against RSV can protect babies from the virus that kills nearly 100,000 children worldwide each year.

Most of those children are under five years old, with 98% living in low and middle-income countries that have fewer healthcare resources to treat the infection.

Pfizer has yet to take a number of steps needed to make the vaccine available in developing countries, according to global health officials and the company.

They include seeking approval for the shot from the WHO, whose authorization is used by governments that don't have a strong domestic drugs regulator, and by nonprofits - such as the global vaccine alliance Gavi - that fund purchases in low-income countries.

Early discussions with the WHO are underway, but the process will be complicated because Pfizer's RSV vaccine is not designed for syringes that automatically disable after one injection, which are preferred by the health agency for safety reasons to prevent reuse.

In addition, the RSV shot is currently only available in a single-dose vial. In lower-resource settings, multi-dose vials are often used because they are cheaper and simpler to distribute.

Multi-dose vials require the addition of a preservative, which means the manufacturer needs to run new trials to ensure there is no impact on the vaccine’s safety or efficacy.

The Gates grant will be used to fund trials of a multi-dose vial, which are likely to begin next year and may take another year to complete, a source with knowledge of the plans said.

A Gates Foundation official said that the investment in the RSV program represented a learning process for the organization and the company on the steps needed for more equitable access.

They have already reached a partnership for a maternal vaccine in earlier stages of development against Group B Streptococcus, which kills around 90,000 babies a year and causes 46,000 stillbirths, over half of them in sub-Saharan Africa.

"We very much want to see that time gap [between high and low-income countries] reduced, such that there's no space between the two," said Padmini Srikantiah, lead on the RSV vaccine launch at Gates.

She still expects it to be several years before the RSV vaccine is launched in lower-income countries.

"I think we're doing better than in years before," she said. "It's a process."

(This story has been corrected to change the approval date in the US to August, not May, in paragraph 4)


April 26 (Reuters) - The U.S. health regulator on Wednesday approved Seres Therapeutics Inc's (MCRB.O) pill for treating a type of bacterial infection, giving an easier and standardized option to patients who often have to rely on individual donors for fecal transplants.

The drug, branded as Vowst, was approved for treating 18 years and older for recurrent Clostridioides difficile (C. diff) infections (CDI), generally caused by prolonged use of antibiotics, which wipe out friendly colon bacteria and can cause potentially fatal diarrhea and inflammation of the colon.

Vowst consists of a certain kind of bacteria made by purifying fecal matter derived from healthy people, while fecal transplants are donated by healthy volunteers and are not purified.

The approval for the drug was based on its late-stage trials, in which treatment with the drug helped patients get rid of recurrent CDI.

"The efficacy is so good, I don't know that there's much room for others to compete with this drug," Oppenheimer analyst Mark Breidenbach said before the approval.

"""

response = generate(prompt + article)
print(response)

"Pfizer has made a commitment to deliver critical new medicines more quickly in low-income nations, but its first such vaccine effort faces hurdles likely to delay distribution in poorer countries by several years. The company received a $28 million grant from the Bill and Melinda Gates Foundation in September to support the launch of the respiratory syncytial virus vaccine in poorer countries, where RSV is much more likely to be lethal for very young children. Pfizer has yet to take a number of steps needed to make the vaccine available in developing countries, including seeking approval for the shot from the WHO, whose authorization is used by governments that don't have a strong domestic drugs regulator, and by nonprofits - such as the global vaccine alliance Gavi. The company has already reached a partnership for a maternal vaccine in earlier stages of development against Group B Streptococcus, which kills around 90,000 babies a year and causes 46,000 stillbirths, over half of them

In [13]:
article = """
Alzheimer’s disease treatment stocks are focused on Alzheimer’s disease, a degenerative brain disorder that results in declining memory and thinking skills and typically affects people in their mid-60s.According to the Alzheimer’s Association, neurons in other areas of the brain also begin to deteriorate as Alzheimer’s disease gets worse, resulting in the loss of basic human functions and overall cognitive impairment.This condition affects more than 6.7 million people in the US alone; it’s also the most common form of dementia and is the sixth leading cause of death in America. Treatments are available to alleviate Alzheimer’s disease symptoms, but because they do not affect the underlying causes of this neurodegenerative disease, they’re only a bandaid solution.Therapies approved by the US Food and Drug Administration (FDA) include: rivastigmine by Novartis (NYSE:NVS); galantamine, developed by Janssen, a division of Johnson & Johnson (NYSE:JNJ); donepezil by Pfizer (NYSE:PFE); and AbbVie's (NYSE:ABBV) memantine.Since there is no cure for Alzheimer’s disease, death is often the result for patients as the ailment devastates the brain. And unfortunately, Alzheimer’s disease is rising in prevalence — a report from Research and Markets suggests that the global Alzheimer’s disease treatment market will be worth a significant US$9.64 billion by 2028 as more patients need treatment, and as more investments are made in biomarkers for diagnosis and drug development.Here the Investing News Network takes a look at a few of the biggest Alzheimer’s disease treatment stocks on the NASDAQ. Companies are listed in order of market cap from largest to smallest, and all data was current as of April 19, 2023.. Biogen (NASDAQ:BIIB) Company Profile.Market cap: US$42.27 billion; current share price: US$292.80.Massachusetts-based Biogen, a pioneer in the field of neuroscience, is developing, manufacturing and marketing therapies aimed at treating serious neurological, neurodegenerative, autoimmune and rare diseases.The global biotechnology firm’s research areas include Alzheimer's disease and dementia. However, Biogen’s launch of its FDA-approved Alzheimer’s disease drug Aduhelm faced a lot of pushback in 2022, both from the market and from Congress, over what was viewed as a hasty fast-track approval process and exorbitant costs to patients.In early 2023, Biogen is looking to give it another go, this time with LEQEMBI (lecanemab-irmb) for the treatment of Alzheimer's disease, which the FDA has approved under its accelerated approval pathway. The drug was jointly developed by Biogen and Tokyo-based pharmaceutical company Eisai (OTC Pink:ESALF,TSE:4523). Intra-Cellular Therapies (NASDAQ:ITCI).Company Profile.Market cap: US$5.96 billion; current share price: US$62.59.As its name suggests, biopharma company Intra-Cellular Therapies is taking an intracellular approach to developing therapies for patients with complex psychiatric and neurologic diseases, including Parkinson’s disease and Alzheimer’s disease.Intra-Cellular Therapies is currently evaluating ITI-1284-ODT-SL, a deuterated form of lumateperone that comes in an oral disintegrating tablet; ITI-1284-ODT-SL is in Phase 1 studies, including drug-drug interaction studies. The company expects to commence Phase 2 clinical trials in agitation in patients with probable Alzheimer’s disease in 2023. 3. Axsome Therapeutics (NASDAQ:AXSM).Company Profile.Market cap: US$3.16 billion; current share price: US$72.64.New York-based Axsome Therapeutics is developing and commercializing a portfolio of novel therapies for disorders of the central nervous system with limited treatment options. Products currently on the market from the biopharmaceutical company include Auvelity, a rapid-acting oral antidepressant for the treatment of major depressive disorder; and Sunosi, a once-daily prescription medicine to treat excessive daytime sleepiness due to obstructive sleep apnea or narcolepsy.Axsome’s product pipeline includes AXS-05, a NMDA receptor antagonist with multimodal activity. It has breakthrough therapy designation from the FDA and is currently in Phase 3 clinical studies for the treatment of Alzheimer’s disease agitation. 
"""

response = generate(prompt + article)
print(response)

"The article discusses the importance of Alzheimer's disease treatment stocks in the health and pharmaceutical industry. The article highlights the prevalence of Alzheimer's disease in the US and the growing global Alzheimer's disease treatment market. The article also highlights the challenges faced by Biogen and Intra-Cellular Therapies in developing therapies for Alzheimer's disease. The article also discusses the potential impact of Axsome Therapeutics on the market and the potential for its product pipeline to be successful. The article concludes by highlighting the potential for these companies to grow and provide investors with potential returns.\n"

In [14]:
article = """
A Mubadala Investment Co.-backed healthcare company said it agreed to acquire kidney-care provider Diaverum, extending the Abu Dhabi sovereign-wealth fund’s bet on healthcare.
Diaverum’s operations include 40 dialysis facilities in Saudi Arabia where Diaverum operates under a five-year contract with the Saudi Ministry of Health, according to the company’s website.
The abortion pill mifepristone is either banned or restricted to varying degrees in 27 states despite a Supreme Court decision that — for now — maintains Food and Drug Administration regulations allowing easy access to the medication.
The Supreme Court, acting on an emergency basis, last week blocked lower federal court orders that had imposed severe restrictions on mifepristone even in some states where abortion remains legal.
Patients previously were required by the FDA to obtain mifepristone in person from a health-care provider or take the medication under their supervision.
While the Supreme Court’s decision at least temporarily maintains access to mifepristone in states that support broad access to abortion, it will have less or no impact in conservative states that banned the procedure since last summer, when the high court overturned the half-century-old federal constitutional right to abortion.
Kaiser Permanente, a large California-based health care group with 39 hospitals and 24,000 doctors, said on Wednesday that it expected to acquire Geisinger Health, a smaller East Coast group, in an effort to develop a new company that would operate nonprofit community health systems.
While Mr. Adams did not say what other health systems he might be talking to regarding acquisitions, Kaiser said it hoped to invest $5 billion in Risant over the next five years, in addition to its spending on Kaiser’s core operations.
The creation of Risant Health represents an opportunity for Kaiser, which had $95 billion in revenue last year, to become an even bigger and more influential organization by working with other hospital groups and health plans.
In keeping with Kaiser’s model, the community health systems under Risant would invest in technology and preventive care to keep patients healthy, so they would need less expensive specialty and hospital care, Mr. Adams said.
“Covid has really shown not having integrated, value-based relationships puts our health systems and our communities at risk,” Mr. Adams said.
While Geisinger has long focused on improving care, Dr. Ryu said the health system would benefit from Kaiser’s ability to invest in the kind of technology and preventive care necessary to keep people healthier.
And unfortunately, Alzheimer’s disease is rising in prevalence — a report from Research and Markets suggests that the global Alzheimer’s disease treatment market will be worth a significant US$9.64 billion by 2028 as more patients need treatment, and as more investments are made in biomarkers for diagnosis and drug development.Here the Investing News Network takes a look at a few of the biggest Alzheimer’s disease treatment stocks on the NASDAQ.
Biogen (NASDAQ:BIIB) Company Profile.Market cap: US$42.27 billion; current share price: US$292.80.Massachusetts-based Biogen, a pioneer in the field of neuroscience, is developing, manufacturing and marketing therapies aimed at treating serious neurological, neurodegenerative, autoimmune and rare diseases.The global biotechnology firm’s research areas include Alzheimer's disease and dementia.
Intra-Cellular Therapies (NASDAQ:ITCI).Company Profile.Market cap: US$5.96 billion; current share price: US$62.59.As its name suggests, biopharma company Intra-Cellular Therapies is taking an intracellular approach to developing therapies for patients with complex psychiatric and neurologic diseases, including Parkinson’s disease and Alzheimer’s disease.Intra-Cellular Therapies is currently evaluating ITI-1284-ODT-SL, a deuterated form of lumateperone that comes in an oral disintegrating tablet; ITI-1284-ODT-SL is in Phase 1 studies, including drug-drug interaction studies.
3. Axsome Therapeutics (NASDAQ:AXSM).Company Profile.Market cap: US$3.16 billion; current share price: US$72.64.New York-based Axsome Therapeutics is developing and commercializing a portfolio of novel therapies for disorders of the central nervous system with limited treatment options.
"""

response = generate(prompt + article)
print(response)

"The article discusses several developments in the healthcare and pharmaceutical industry, including the acquisition of kidney-care provider Diaverum by Mubadala Investment Co., the Supreme Court decision on mifepristone, the creation of Risant Health, and the growing prevalence of Alzheimer's disease. The article also provides information on the biggest Alzheimer's disease treatment stocks on the NASDAQ, including Biogen, Intra-Cellular Therapies, and Axsome Therapeutics.\n"

In [15]:
article = """
A Mubadala Investment Co.-backed healthcare company said it agreed to acquire kidney-care provider Diaverum, extending the Abu Dhabi sovereign-wealth fund’s bet on healthcare.
Diaverum’s operations include 40 dialysis facilities in Saudi Arabia where Diaverum operates under a five-year contract with the Saudi Ministry of Health, according to the company’s website.
The abortion pill mifepristone is either banned or restricted to varying degrees in 27 states despite a Supreme Court decision that — for now — maintains Food and Drug Administration regulations allowing easy access to the medication.
The Supreme Court, acting on an emergency basis, last week blocked lower federal court orders that had imposed severe restrictions on mifepristone even in some states where abortion remains legal.
Patients previously were required by the FDA to obtain mifepristone in person from a health-care provider or take the medication under their supervision.
While the Supreme Court’s decision at least temporarily maintains access to mifepristone in states that support broad access to abortion, it will have less or no impact in conservative states that banned the procedure since last summer, when the high court overturned the half-century-old federal constitutional right to abortion.
Kaiser Permanente, a large California-based health care group with 39 hospitals and 24,000 doctors, said on Wednesday that it expected to acquire Geisinger Health, a smaller East Coast group, in an effort to develop a new company that would operate nonprofit community health systems.
While Mr. Adams did not say what other health systems he might be talking to regarding acquisitions, Kaiser said it hoped to invest $5 billion in Risant over the next five years, in addition to its spending on Kaiser’s core operations.
The creation of Risant Health represents an opportunity for Kaiser, which had $95 billion in revenue last year, to become an even bigger and more influential organization by working with other hospital groups and health plans.
In keeping with Kaiser’s model, the community health systems under Risant would invest in technology and preventive care to keep patients healthy, so they would need less expensive specialty and hospital care, Mr. Adams said.
“Covid has really shown not having integrated, value-based relationships puts our health systems and our communities at risk,” Mr. Adams said.
While Geisinger has long focused on improving care, Dr. Ryu said the health system would benefit from Kaiser’s ability to invest in the kind of technology and preventive care necessary to keep people healthier.
And unfortunately, Alzheimer’s disease is rising in prevalence — a report from Research and Markets suggests that the global Alzheimer’s disease treatment market will be worth a significant US$9.64 billion by 2028 as more patients need treatment, and as more investments are made in biomarkers for diagnosis and drug development.Here the Investing News Network takes a look at a few of the biggest Alzheimer’s disease treatment stocks on the NASDAQ.
Biogen (NASDAQ:BIIB) Company Profile.Market cap: US$42.27 billion; current share price: US$292.80.Massachusetts-based Biogen, a pioneer in the field of neuroscience, is developing, manufacturing and marketing therapies aimed at treating serious neurological, neurodegenerative, autoimmune and rare diseases.The global biotechnology firm’s research areas include Alzheimer's disease and dementia.
Intra-Cellular Therapies (NASDAQ:ITCI).Company Profile.Market cap: US$5.96 billion; current share price: US$62.59.As its name suggests, biopharma company Intra-Cellular Therapies is taking an intracellular approach to developing therapies for patients with complex psychiatric and neurologic diseases, including Parkinson’s disease and Alzheimer’s disease.Intra-Cellular Therapies is currently evaluating ITI-1284-ODT-SL, a deuterated form of lumateperone that comes in an oral disintegrating tablet; ITI-1284-ODT-SL is in Phase 1 studies, including drug-drug interaction studies.
3. Axsome Therapeutics (NASDAQ:AXSM).Company Profile.Market cap: US$3.16 billion; current share price: US$72.64.New York-based Axsome Therapeutics is developing and commercializing a portfolio of novel therapies for disorders of the central nervous system with limited treatment options.
"""

prompt = "You are an equity researcher specialized in the health and pharmaceutical industry. Write a summary for the following sentences extracted from news articles. Focus on the information that will impact investment decisions on the companies mentioned."

response = generate(prompt + article)
print(response)

"The healthcare industry is undergoing significant changes as a result of the COVID-19 pandemic. Mubadala Investment Co. has announced its intention to acquire kidney-care provider Diaverum, which operates 40 dialysis facilities in Saudi Arabia. The Supreme Court has temporarily maintained FDA regulations allowing easy access to the abortion pill mifepristone, but it will have less or no impact in conservative states that ban the procedure. Kaiser Permanente is expected to acquire Geisinger Health in an effort to develop a new company that would operate nonprofit community health systems. The global Alzheimer's disease treatment market is expected to be worth a significant US$9.64 billion by 2028, and there are several stocks on the NASDAQ that are developing therapies for the disease. Biogen, Intra-Cellular Therapies, and Axsome Therapeutics are among the biggest Alzheimer's disease treatment stocks on the NASDAQ.\n"

In [16]:
article = """
A Mubadala Investment Co.-backed healthcare company said it agreed to acquire kidney-care provider Diaverum, extending the Abu Dhabi sovereign-wealth fund’s bet on healthcare. M42, a newly launched healthcare operator owned by Mubadala and a partner company, G42, is acquiring Diaverum from U.K.-based buyout firm Bridgepoint Group PLC. BPT 0.17%increase; green up pointing triangle The deal value wasn’t disclosed. The Wall Street Journal earlier reported the transaction values the business at more than $2 billion including debt, according to people familiar with the matter. M42 said it expects to close the deal later this year. A pre-markets primer packed with news, trends and ideas. Plus, up-to-the-minute market data. Diaverum, based in Sweden, oversees a global network of 440 dialysis clinics, almost triple the 155 facilities Bridgepoint inherited when it bought the business in 2007 for an undisclosed amount. Diaverum has also expanded geographically over that period, operating in 23 countries, up from 15.  Dialysis is a procedure to remove waste products and excess fluid from the blood when the kidneys stop working properly. It often involves diverting blood to a machine to be cleaned. The size of the dialysis market globally is expected to grow in coming years because of aging populations and the increased incidence of kidney disease related to diabetes and other conditions. Diaverum’s operations include 40 dialysis facilities in Saudi Arabia where Diaverum operates under a five-year contract with the Saudi Ministry of Health, according to the company’s website. Confidence in winning renewal of the license in 2024 was key to the deal, which gave M42 an advantage over other bidders because of its existing base in the region and Mubadala’s backing, said some bankers following the sale process.  Mubadala is the investment arm for the Abu Dhabi government. It oversees $284 billion in assets and invests across a range of industries including business services, energy, financials, healthcare and industrials. Bridgepoint manages more than €37 billion, equivalent to about $40.6 billion of assets, focusing its investments in industrials, business and financial services, consumer and healthcare, according to the firm’s website. Bridgepoint attempted to list Diaverum in an initial public offering in 2020. It withdrew the IPO after not achieving a high-enough valuation.
"""

prompt = "You are an equity researcher specialized in the health and pharmaceutical industry. Write a summary for the following article. Focus on the information that will impact investment decisions on the companies mentioned."
response = generate(prompt + article)
print(response)

"The article discusses the acquisition of Diaverum, a Swedish dialysis provider, by M42, a newly launched healthcare operator owned by Mubadala Investment Co. and a partner company, G42. The deal value of the acquisition is not disclosed, but the Wall Street Journal earlier reported the transaction values the business at more than $2 billion including debt. M42 expects to close the deal later this year.\nDiaverum is a Swedish dialysis provider with a global network of 440 dialysis clinics. The company operates in 23 countries and has 40 dialysis facilities in Saudi Arabia, where it operates under a five-year contract with the Saudi Ministry of Health. The size of the dialysis market globally is expected to grow in coming years due to aging populations and the increased incidence of kidney disease related to diabetes and other conditions.\nM42 has an existing base in the region and Mubadala's backing, which gave it an advantage over other bidders in the sale process. Bridgepoint Group a

In [17]:
article = """
April 26 (Reuters) - The U.S. health regulator on Wednesday approved Seres Therapeutics Inc's (MCRB.O) pill for treating a type of bacterial infection, giving an easier and standardized option to patients who often have to rely on individual donors for fecal transplants.

The drug, branded as Vowst, was approved for treating 18 years and older for recurrent Clostridioides difficile (C. diff) infections (CDI), generally caused by prolonged use of antibiotics, which wipe out friendly colon bacteria and can cause potentially fatal diarrhea and inflammation of the colon.

Vowst consists of a certain kind of bacteria made by purifying fecal matter derived from healthy people, while fecal transplants are donated by healthy volunteers and are not purified.

The approval for the drug was based on its late-stage trials, in which treatment with the drug helped patients get rid of recurrent CDI.

"The efficacy is so good, I don't know that there's much room for others to compete with this drug," Oppenheimer analyst Mark Breidenbach said before the approval.

Seres' drug will compete with Switzerland-based Ferring Pharmaceuticals' fecal transplant-based therapy, which is delivered through an enema.

Physicians think Seres' pill could replace fecal microbiota transplants in patients, TD Cowen analyst Joseph Thome said ahead of the decision. He said the drug could be priced close to $20,000 per course, expecting peak sales of $750 million in the U.S. in 2033.

Government data estimates that C. diff causes nearly half a million infections in the United States annually.

Seres entered into a deal with Nestle's health science unit in 2021 to jointly commercialize the drug in the United States.
"""

prompt = "You are an equity researcher specialized in the health and pharmaceutical industry. Write a summary for the following article. Focus on the information that will impact investment decisions on the companies mentioned."

response = generate(prompt + article)
print(response)

"The U.S. health regulator has approved Seres Therapeutics' (MCRB.O) pill for treating a type of bacterial infection, giving patients an easier and standardized option to fecal transplants. The drug, branded as Vowst, is made by purifying fecal matter derived from healthy people, while fecal transplants are donated by healthy volunteers and are not purified. The approval for the drug was based on its late-stage trials, in which treatment with the drug helped patients get rid of recurrent CDI. Seres' drug will compete with Switzerland-based Ferring Pharmaceuticals' fecal transplant-based therapy, which is delivered through an enema. Physicians think Seres' pill could replace fecal microbiota transplants in patients, and analysts expect peak sales of $750 million in the U.S. in 2033. The drug could be priced close to $20,000 per course, with peak sales of $750 million in the U.S. expected in 2033. Seres entered into a deal with Nestle's health science unit in 2021 to jointly commercializ

In [18]:
article = """
April 26 (Reuters) - The U.S. health regulator on Wednesday approved Seres Therapeutics Inc's (MCRB.O) pill for treating a type of bacterial infection, giving an easier and standardized option to patients who often have to rely on individual donors for fecal transplants.

The drug, branded as Vowst, was approved for treating 18 years and older for recurrent Clostridioides difficile (C. diff) infections (CDI), generally caused by prolonged use of antibiotics, which wipe out friendly colon bacteria and can cause potentially fatal diarrhea and inflammation of the colon.

Vowst consists of a certain kind of bacteria made by purifying fecal matter derived from healthy people, while fecal transplants are donated by healthy volunteers and are not purified.

The approval for the drug was based on its late-stage trials, in which treatment with the drug helped patients get rid of recurrent CDI.

"The efficacy is so good, I don't know that there's much room for others to compete with this drug," Oppenheimer analyst Mark Breidenbach said before the approval.

Seres' drug will compete with Switzerland-based Ferring Pharmaceuticals' fecal transplant-based therapy, which is delivered through an enema.

Physicians think Seres' pill could replace fecal microbiota transplants in patients, TD Cowen analyst Joseph Thome said ahead of the decision. He said the drug could be priced close to $20,000 per course, expecting peak sales of $750 million in the U.S. in 2033.

Government data estimates that C. diff causes nearly half a million infections in the United States annually.

Seres entered into a deal with Nestle's health science unit in 2021 to jointly commercialize the drug in the United States.
"""

prompt = "Rewrite the following article so that all pronouns are replaced with the entity it is referring to."

response = generate(prompt + article)
print(response)

'April 26 (Reuters) - The U.S. health regulator on Wednesday approved Seres Therapeutics Inc\'s (MCRB.O) pill for treating a type of bacterial infection, giving an easier and standardized option to patients who often have to rely on individual donors for fecal transplants. The drug, branded as Vowst, was approved for treating 18 years and older for recurrent Clostridioides difficile (C. diff) infections (CDI), generally caused by prolonged use of antibiotics, which wipe out friendly colon bacteria and can cause potentially fatal diarrhea and inflammation of the colon. Vowst consists of a certain kind of bacteria made by purifying fecal matter derived from healthy people, while fecal transplants are donated by healthy volunteers and are not purified. The approval for the drug was based on its late-stage trials, in which treatment with the drug helped patients get rid of recurrent CDI. "The efficacy is so good, I don\'t know that there\'s much room for others to compete with this drug," 